<a href="https://colab.research.google.com/github/tedhwang007/CS114.L11.KHCL/blob/master/Annotation%20Tool/model_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install seqeval

     |████████████████████████████████| 43 kB 742 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=2932ed97eef17fa94b589fe06fd8fa34e796c7f59621ee76fdbfda866f408f9d
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
!pip install scikit-learn==0.23.0

     |████████████████████████████████| 7.3 MB 4.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.0 which is incompatible.


In [ ]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 743 kB 5.6 MB/s 


In [ ]:
import pandas as pd
import numpy as np
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
!git clone https://github.com/VinAIResearch/PhoNER_COVID19/

Cloning into 'PhoNER_COVID19'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 44 (delta 14), reused 30 (delta 12), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [ ]:
dir_train_data = 'PhoNER_COVID19/data/word/train_word.conll'
dir_dev_data = 'PhoNER_COVID19/data/word/dev_word.conll'
dir_test_data = 'PhoNER_COVID19/data/word/test_word.conll'

In [ ]:
def convert_data_1(dir):
  raw = open(dir_train_data, 'r').readlines()
  all_x = []
  point = []
  for line in raw:
      stripped_line = line.strip().split(' ')
      point.append(stripped_line)
      if line == '\n':
          all_x.append(point[:-1])
          point = []
  all_x = all_x[:-1]
  return all_x
train = convert_data_1(dir_train_data)
dev = convert_data_1(dir_dev_data)
test = convert_data_1(dir_test_data)

In [ ]:
def convert_data_2(filename):
  data = []
  for i in range(len(filename)):
    word_ner = []
    for j in range(len(filename[i])):
      x= tuple((filename[i][j][0],filename[i][j][1]))
      word_ner.append(x)
    data.append(word_ner)
  return data

In [ ]:
Train_data = convert_data_2(train)
Dev_data = convert_data_2(dev)
Test_data = convert_data_2(test)

# **Model**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
import pickle


In [ ]:
def prepareData(tagged_sentences):
    X,y=[],[]
    for sentences in tagged_sentences:
        X.append([word for word,tag in sentences])
        y.append([tag for word,tag in sentences])
    return X,y

In [ ]:
X_train,y_train = prepareData(Train_data[:1000])
X_test,y_test = prepareData(Test_data)
print(y_train)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORGANIZATION', 'I-ORGANIZATION', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SYMPTOM_AND_DISEASE', 'I-SYMPTOM_AND_DISEASE', 'O', 'B-SYMPTOM_AND_DISEASE', 'I-SYMPTOM_AND_DISEASE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'O', 'B-LOCATION', 'O', 'O', 'O', 'O', 'B-LOCATION', 'O', 'B-DATE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B-PATIENT_ID', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PATIENT_ID', 'O', 'O', 'B-AGE', 'O', 'O', 'O', 'B-ORGANIZATION', 'I-ORGANIZATION', 'O', 'O', 'O', 'O', 'B-DATE', 'O'], ['O', 'O', 'B-PATIENT_ID', 'B-NAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [ ]:
crf = CRF()
crf.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(keep_tempfiles=None)

In [ ]:
y_pred = crf.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))
print(f1_score(y_test,y_pred))

                     precision    recall  f1-score   support

                AGE       0.57      0.03      0.06       682
               DATE       0.94      0.91      0.92      2547
             GENDER       0.68      0.48      0.56       542
                JOB       0.46      0.21      0.29       205
           LOCATION       0.72      0.64      0.67      5398
               NAME       0.46      0.11      0.18       349
       ORGANIZATION       0.69      0.54      0.61      1137
         PATIENT_ID       0.71      0.81      0.76      3240
SYMPTOM_AND_DISEASE       0.59      0.15      0.23      1439
     TRANSPORTATION       0.99      0.63      0.77       226

          micro avg       0.75      0.62      0.68     15765
          macro avg       0.68      0.45      0.51     15765
       weighted avg       0.72      0.62      0.64     15765

0.6761513844441351


In [ ]:
filename = '/content/drive/MyDrive/CS321_PROJECT/Data/crf_model_1000.sav'
pickle.dump(crf, open(filename, 'wb'))